In [2]:
!pip freeze #| grep scikit-learn

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import sklearn 
print('sklearn: {}'. format(sklearn. __version__))

sklearn: 1.1.1


In [5]:
import pickle
import pandas as pd

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

c:\Users\hansk\anaconda3\envs\mlflow\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\hansk\anaconda3\envs\mlflow\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [9]:
df = read_data('fhv_tripdata_2021-02.parquet')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [11]:
y_pred.mean()

16.191691679979066

In [45]:
year = 2021
month = 2
taxi_type = 'fhv'
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['predictions'] = y_pred

In [46]:
df_result = df[['ride_id', 'predictions']].copy()
#df_result['predictions'] = y_pred

In [48]:
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'
#output_file

'output/fhv/2021-02.parquet'

In [49]:
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)